# 1. **Preparing Process**

---

In [4]:
import random
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import Dataset, DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from resnet import ResNet
from resnet_liq import ResNet as ResNetLiQ
from time import time
from contextlib import contextmanager
import os
from torch.cuda import max_memory_allocated

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.backends.cudnn.benchmark = True

In [5]:
@contextmanager
def print_and_log(log_filename):
    log_file = open(log_filename, 'w')
    def log(message):
        print(message)
        log_file.write(message + '\n')
        log_file.flush()
    yield log
    log_file.close()

class ApplyTransform(Dataset):
    def __init__(self, dataset, transform):
        self.dataset = dataset
        self.transform = transform

    def __getitem__(self, idx):
        x, y = self.dataset[idx]
        return self.transform(x), y

    def __len__(self):
        return len(self.dataset)

# normalize = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
normalize = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

TRANSFORM_TRAIN = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    normalize,
])
TRANSFORM_TEST = transforms.Compose([
    transforms.ToTensor(),
    normalize,
])

TRAIN_SET = datasets.CIFAR10(root='./data', train=True, download=True, transform=TRANSFORM_TRAIN)
TEST_SET = datasets.CIFAR10(root='./data', train=False, download=True, transform=TRANSFORM_TEST)

100%|██████████| 170498071/170498071 [00:04<00:00, 41635369.70it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [6]:
def test(model: nn.Module, loader: DataLoader):
    criterion = nn.CrossEntropyLoss()
    test_loss = 0
    test_correct = 0
    test_total = 0
    start_time = time()
    torch.cuda.reset_max_memory_allocated(DEVICE)

    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            test_correct += (outputs.argmax(1) == targets).sum().item()
            test_total += targets.size(0)

    latency = time() - start_time
    max_memory_usage = max_memory_allocated(DEVICE) / (1024 ** 2)

    test_loss /= len(loader)
    test_acc = test_correct / test_total
    return test_loss, test_acc, latency, max_memory_usage

In [7]:
def train(w_nbits, a_nbits, linear=False, lr=0.1, weight_decay=1e-3,
          optimizer_type='sgd', epochs=200, batch_size=128):

    torch.cuda.reset_max_memory_allocated(DEVICE)

    if linear:
        model = ResNetLiQ(w_nbits=w_nbits, a_nbits=a_nbits).to(DEVICE)
    else:
        model = ResNet(w_nbits=w_nbits, a_nbits=a_nbits).to(DEVICE)

    ckpt_base_filename = (
        'resnet20_cifar'
        + ('_linear' if linear else '')
        + ('' if w_nbits is None else f'_wq{w_nbits}')
        + ('' if a_nbits is None else f'_aq{a_nbits}')
        + '.pt'
    )
    log_filename = ckpt_base_filename.replace('.pt', '.txt')
    last_saved_filename = None

    train_loader = DataLoader(TRAIN_SET, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(TEST_SET, batch_size=batch_size, shuffle=False)

    criterion = nn.CrossEntropyLoss()

    if optimizer_type.lower() == 'sgd':
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
    elif optimizer_type.lower() == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=lr)
    elif optimizer_type.lower() == 'rmsprop':
        optimizer = optim.RMSprop(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif optimizer_type.lower() == 'adagrad':
        optimizer = optim.Adagrad(model.parameters(), lr=lr, weight_decay=weight_decay)
    else:
        raise ValueError(f"Unsupported optimizer type: {optimizer_type}")

    scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[50, 100, 150], gamma=0.1)

    best_test_acc = 0.0
    save_threshold_epoch = min(50, epochs // 3)


    with print_and_log(log_filename) as log:
        log(f'\nQuantization: weight={w_nbits} activation={a_nbits} linear={linear}, Using device: {DEVICE}')

        for epoch in range(epochs):
            log(f'\nEpoch {epoch+1}')
            model.train()

            train_loss = 0
            train_correct = 0
            train_total = 0

            start_time = time()

            for inputs, targets in tqdm(train_loader, desc='Train', unit='batch', ascii=True, dynamic_ncols=True):
                inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()

                train_loss += loss.item()
                train_correct += (outputs.argmax(1) == targets).sum().item()
                train_total += targets.size(0)

            train_loss /= len(train_loader)
            train_acc = train_correct / train_total

            train_latency = time() - start_time
            max_memory_usage_during_training = max_memory_allocated(DEVICE) / (1024 ** 2)

            log(f'LR: {optimizer.param_groups[0]["lr"]:.4e}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Train Latency: {train_latency:.2f}s, Memory Usage: {max_memory_usage_during_training:.2f}MB')

            model.eval()
            test_loss, test_acc, test_latency, test_memory_usage = test(model, test_loader)
            log(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}, Test Latency: {test_latency:.2f}s, Test Memory Usage: {test_memory_usage:.2f}MB')

            scheduler.step()

            if epoch >= save_threshold_epoch:
                if test_acc > best_test_acc:
                    best_test_acc = test_acc
                    print('Saving best model ...')
                    if last_saved_filename is not None and last_saved_filename != ckpt_base_filename:
                        os.remove(last_saved_filename)
                    ckpt_filename = f'epoch_{epoch+1}_' + ckpt_base_filename
                    torch.save(model.state_dict(), ckpt_filename)
                    last_saved_filename = ckpt_filename

    print('Saving final epoch model ...')
    torch.save(model.state_dict(), ckpt_base_filename)
    return ckpt_base_filename

In [8]:
from lq_layers import LQConv, LQActiv
from liq_layers import LiQConv, LiQActiv
import numpy as np
import matplotlib.pyplot as plt
import re
import os


def parse_quantization_bits_from_filename(filename):
    match_w = re.search(r'_wq(\d+)', filename)
    match_a = re.search(r'_aq(\d+)', filename)

    w_nbits = int(match_w.group(1)) if match_w else None
    a_nbits = int(match_a.group(1)) if match_a else None
    base_filename = os.path.splitext(os.path.basename(filename))[0]

    contains_linear = 'linear' in base_filename

    print('weight bits:', w_nbits, 'activation bits:', a_nbits, 'is linear:', contains_linear, 'base filename:', base_filename)

    return w_nbits, a_nbits, contains_linear, base_filename

def load_model(model_path, num_layers, num_classes, w_nbits, a_nbits, device):
    model = ResNet(w_nbits=w_nbits, a_nbits=a_nbits)
    model.to(device)

    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    return model

def collect_activations_and_quantized_weights(model, dataloader, device,linear = False):
    activations = {}
    quantized_weights = {}

    def get_quantized_weight(name, layer):
        def hook(model, input, output):
            q_weight = layer.lq.apply(layer.conv.weight, layer.basis, False)[0]
            quantized_weights[name] = q_weight.detach().cpu().numpy()
        return hook

    def get_activation(name, layer):
      def hook(model, input, output):
          activations[name] = output.detach().cpu().numpy()
      return hook

    for name, layer in model.named_modules():
        if linear:
            if isinstance(layer, LiQConv):
                layer.register_forward_hook(get_quantized_weight(name, layer))
            if isinstance(layer, LiQActiv):
                layer.register_forward_hook(get_activation(name, layer))
        else:
            if isinstance(layer, LQConv):
                layer.register_forward_hook(get_quantized_weight(name, layer))
            if isinstance(layer, LQActiv):
                layer.register_forward_hook(get_activation(name, layer))

    with torch.no_grad():
        for inputs, _ in dataloader:
            inputs = inputs.to(device)
            _ = model(inputs)

    return activations, quantized_weights

def plot_distributions_side_by_side(activations, quantized_weights, base_filename):
    sorted_weight_keys = sorted(quantized_weights.keys())
    sorted_activation_keys = sorted(activations.keys())

    num_layers = max(len(sorted_weight_keys), len(sorted_activation_keys))
    fig, axes = plt.subplots(num_layers, 2, figsize=(12, 6 * num_layers))

    output_directory = f"./output/{base_filename}"
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    for i, (weight_key, activation_key) in enumerate(zip(sorted_weight_keys, sorted_activation_keys)):
        ax_w = axes[i][0] if num_layers > 1 else axes[0]
        ax_a = axes[i][1] if num_layers > 1 else axes[1]
        ax_w.hist(quantized_weights[weight_key].ravel(), bins=50, alpha=0.75)
        ax_w.set_title(f'Quantized Weight Distribution: {weight_key}')
        ax_w.set_xlabel('Weight Values')
        ax_w.set_ylabel('Frequency')

        ax_a.hist(activations[activation_key].ravel(), bins=50, alpha=0.75)
        ax_a.set_title(f'Activation Distribution: {activation_key}')
        ax_a.set_xlabel('Activation Values')
        ax_a.set_ylabel('Frequency')

        plot_filename = f"{output_directory}/{base_filename}_{weight_key}_and_{activation_key}.png"
        plt.savefig(plot_filename)
        print(f"Saved plot as {plot_filename}")

    plt.close(fig)

def analyze_model(model_filename, device='cuda'):
    w_nbits, a_nbits, linear, base_filename = parse_quantization_bits_from_filename(model_filename)
    num_layers = 20
    num_classes = 10

    model = load_model(model_filename, num_layers, num_classes, w_nbits, a_nbits, device)
    test_loader = DataLoader(TEST_SET, batch_size=64, shuffle=False)

    activations, weights = collect_activations_and_quantized_weights(model, test_loader, device, linear=linear)
    plot_distributions_side_by_side(activations, weights, base_filename)


# 2. **Base Model**

---


In [9]:
model_name = train(w_nbits=None, a_nbits=None, linear=True)

/usr/local/lib/python3.10/dist-packages/torch/cuda/memory.py:330: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(



Quantization: weight=None activation=None linear=True, Using device: cuda

Epoch 1


Train: 100%|##########| 391/391 [00:27<00:00, 14.36batch/s]


LR: 1.0000e-01, Train Loss: 1.6845, Train Acc: 0.3783, Train Latency: 27.23s, Memory Usage: 291.35MB
Test Loss: 1.6865, Test Acc: 0.4535, Test Latency: 2.74s, Test Memory Usage: 61.86MB

Epoch 2


Train: 100%|##########| 391/391 [00:24<00:00, 15.94batch/s]


...

LR: 1.0000e-04, Train Loss: 0.0364, Train Acc: 0.9922, Train Latency: 24.63s, Memory Usage: 212.38MB
Test Loss: 0.2670, Test Acc: 0.9218, Test Latency: 2.73s, Test Memory Usage: 61.86MB

Epoch 199


Train: 100%|##########| 391/391 [00:24<00:00, 15.89batch/s]


LR: 1.0000e-04, Train Loss: 0.0375, Train Acc: 0.9916, Train Latency: 24.60s, Memory Usage: 212.38MB
Test Loss: 0.2667, Test Acc: 0.9211, Test Latency: 2.68s, Test Memory Usage: 61.86MB

Epoch 200


Train: 100%|##########| 391/391 [00:24<00:00, 15.90batch/s]


LR: 1.0000e-04, Train Loss: 0.0358, Train Acc: 0.9928, Train Latency: 24.59s, Memory Usage: 212.38MB
Test Loss: 0.2678, Test Acc: 0.9212, Test Latency: 2.69s, Test Memory Usage: 61.86MB
Saving final epoch model ...


In [10]:
# analyze_model('resnet20_cifarlinear.pt')

# **Weight Quantization**

---

In [11]:
model_name_list = []
for w_nbits in [3, 2, 1]:
    temp_name = train(w_nbits=w_nbits, a_nbits=None, linear=True)
    model_name_list.append(temp_name)


Quantization: weight=3 activation=None linear=True, Using device: cuda

Epoch 1


Train: 100%|##########| 391/391 [00:25<00:00, 15.07batch/s]


LR: 1.0000e-01, Train Loss: 1.8193, Train Acc: 0.3517, Train Latency: 25.95s, Memory Usage: 213.40MB
Test Loss: 1.7265, Test Acc: 0.4345, Test Latency: 2.97s, Test Memory Usage: 61.86MB

Epoch 2


Train: 100%|##########| 391/391 [00:25<00:00, 15.22batch/s]


LR: 1.0000e-01, Train Loss: 1.2687, Train Acc: 0.5382, Train Latency: 25.70s, Memory Usage: 213.40MB
Test Loss: 1.4622, Test Acc: 0.5176, Test Latency: 2.93s, Test Memory Usage: 61.86MB

Epoch 3


...

LR: 1.0000e-04, Train Loss: 0.9647, Train Acc: 0.6538, Train Latency: 25.80s, Memory Usage: 213.40MB
Test Loss: 1.5129, Test Acc: 0.4839, Test Latency: 2.96s, Test Memory Usage: 61.86MB

Epoch 199


Train: 100%|##########| 391/391 [00:25<00:00, 15.04batch/s]


LR: 1.0000e-04, Train Loss: 0.9602, Train Acc: 0.6578, Train Latency: 26.00s, Memory Usage: 213.40MB
Test Loss: 2.6129, Test Acc: 0.3321, Test Latency: 3.05s, Test Memory Usage: 61.86MB

Epoch 200


Train: 100%|##########| 391/391 [00:26<00:00, 14.98batch/s]


LR: 1.0000e-04, Train Loss: 0.9624, Train Acc: 0.6594, Train Latency: 26.11s, Memory Usage: 213.40MB
Test Loss: 1.8899, Test Acc: 0.4371, Test Latency: 2.92s, Test Memory Usage: 61.86MB
Saving final epoch model ...


In [12]:
# for model_name in model_name_list:
#     print('_'*50)
#     analyze_model(model_name)
#     print('_'*50)

# **Weight & Activation**

---

In [13]:
model_name = train(w_nbits=1, a_nbits=2, linear=True)


Quantization: weight=1 activation=2 linear=True, Using device: cuda

Epoch 1


Train: 100%|##########| 391/391 [00:26<00:00, 14.53batch/s]


LR: 1.0000e-01, Train Loss: 5.4831, Train Acc: 0.1959, Train Latency: 26.92s, Memory Usage: 295.40MB
Test Loss: 501.5986, Test Acc: 0.0998, Test Latency: 3.14s, Test Memory Usage: 77.86MB

Epoch 2


Train: 100%|##########| 391/391 [00:26<00:00, 14.51batch/s]


LR: 1.0000e-01, Train Loss: 156.7066, Train Acc: 0.1154, Train Latency: 26.95s, Memory Usage: 295.40MB
Test Loss: 3569.5803, Test Acc: 0.1003, Test Latency: 3.12s, Test Memory Usage: 77.86MB

Epoch 3


...

LR: 1.0000e-04, Train Loss: 2.1793, Train Acc: 0.1859, Train Latency: 26.75s, Memory Usage: 295.40MB
Test Loss: 2.1691, Test Acc: 0.1895, Test Latency: 3.11s, Test Memory Usage: 77.86MB

Epoch 199


Train: 100%|##########| 391/391 [00:26<00:00, 14.66batch/s]


LR: 1.0000e-04, Train Loss: 2.1790, Train Acc: 0.1849, Train Latency: 26.67s, Memory Usage: 295.40MB
Test Loss: 2.2017, Test Acc: 0.1709, Test Latency: 3.11s, Test Memory Usage: 77.86MB

Epoch 200


Train: 100%|##########| 391/391 [00:26<00:00, 14.73batch/s]


LR: 1.0000e-04, Train Loss: 2.1785, Train Acc: 0.1876, Train Latency: 26.54s, Memory Usage: 295.40MB
Test Loss: 2.2510, Test Acc: 0.1892, Test Latency: 3.10s, Test Memory Usage: 77.86MB
Saving final epoch model ...


In [ ]:
# analyze_model(model_name)

In [ ]:
model_name = train(w_nbits=2, a_nbits=2, linear=True)

In [16]:
# analyze_model(model_name)

In [ ]:
model_name = train(w_nbits=2, a_nbits=3, linear=True)

In [ ]:
# analyze_model(model_name)

In [19]:
model_name = train(w_nbits=3, a_nbits=3, linear=True)


Quantization: weight=3 activation=3 linear=True, Using device: cuda

Epoch 1


Train: 100%|##########| 391/391 [00:26<00:00, 14.51batch/s]


LR: 1.0000e-01, Train Loss: 1.7647, Train Acc: 0.3345, Train Latency: 26.95s, Memory Usage: 295.40MB
Test Loss: 1.6442, Test Acc: 0.3826, Test Latency: 3.16s, Test Memory Usage: 77.86MB

Epoch 2


Train: 100%|##########| 391/391 [00:26<00:00, 14.57batch/s]


LR: 1.0000e-01, Train Loss: 1.4296, Train Acc: 0.4768, Train Latency: 26.85s, Memory Usage: 295.40MB
Test Loss: 1.6452, Test Acc: 0.4493, Test Latency: 3.13s, Test Memory Usage: 77.86MB

Epoch 3


...

LR: 1.0000e-04, Train Loss: 0.4479, Train Acc: 0.8455, Train Latency: 27.27s, Memory Usage: 295.40MB
Test Loss: 0.5623, Test Acc: 0.8125, Test Latency: 3.13s, Test Memory Usage: 77.86MB

Epoch 199


Train: 100%|##########| 391/391 [00:27<00:00, 14.38batch/s]


LR: 1.0000e-04, Train Loss: 0.4499, Train Acc: 0.8428, Train Latency: 27.19s, Memory Usage: 295.40MB
Test Loss: 0.5563, Test Acc: 0.8067, Test Latency: 3.15s, Test Memory Usage: 77.86MB

Epoch 200


Train: 100%|##########| 391/391 [00:27<00:00, 14.41batch/s]


LR: 1.0000e-04, Train Loss: 0.4457, Train Acc: 0.8437, Train Latency: 27.14s, Memory Usage: 295.40MB
Test Loss: 0.5351, Test Acc: 0.8193, Test Latency: 3.14s, Test Memory Usage: 77.86MB
Saving final epoch model ...


In [20]:
# analyze_model(model_name)